In [1]:
from __future__ import division

import graphlab as gl
import pandas as pd
import pyLDAvis
import pyLDAvis.graphlab

pyLDAvis.enable_notebook()

This trial license of GraphLab Create assigned to pbreen@uga.edu has expired. Please contact trial@turi.com for licensing options or to request a free non-commercial license for academic use.
This trial license of GraphLab Create assigned to pbreen@uga.edu has expired. Please contact trial@turi.com for licensing options or to request a free non-commercial license for academic use.
This trial license of GraphLab Create assigned to pbreen@uga.edu has expired. Please contact trial@turi.com for licensing options or to request a free non-commercial license for academic use.
This trial license of GraphLab Create assigned to pbreen@uga.edu has expired. Please contact trial@turi.com for licensing options or to request a free non-commercial license for academic use.
This trial license of GraphLab Create assigned to pbreen@uga.edu has expired. Please contact trial@turi.com for licensing options or to request a free non-commercial license for academic use.
This trial license of GraphLab Create as

# Preprocess

In [2]:
# load and concatenate tweet timeline-documents
import json

train = False

user_timelines = dict()
with open('../data/user_timeline_tweets.json') as f_in:
    for l in f_in:
        try:
            twt = json.loads(l)
        except:
            continue
        user_id_str = twt['user']['id_str']
        if user_id_str in user_timelines:
            user_timelines[user_id_str] = user_timelines[user_id_str] + ' ' + twt['text']
        else:
            user_timelines[user_id_str] = twt['text']

IOError: [Errno 2] No such file or directory: '../data/user_timeline_tweets.json'

In [ ]:
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
import re


STOPWORDS = set(['rt', '', 'httpst', 'amp', 'https\'', 'ang',
                'ng', 'tcot', 'ako', 'sa', 'ko', 'rp', 'roc',
                'fe', 'sk', 'holi', 'asa', 'guru', 'yah', 'lista',
                 'hah' 'il', 'seo', 'casa', 'qotd', 'noh']).union(set(STOPWORDS)) \
    .union(set(stopwords.words('spanish'))) \
    .union(set(stopwords.words('portuguese'))) \
    .union(set(stopwords.words('german'))) \
    .union(set(stopwords.words('dutch')))

def clean_text(text):
    clean = re.sub(r'http.*$', '', text)
    clean = re.sub(r'[^a-zA-Z\s]', '', clean)

    clean = clean.lower()
    word_list = clean.split()
    word_list = [w for w in word_list if (w not in STOPWORDS)]

    return word_list

user_clean_timelines = dict()
for user_id_str in user_timelines.keys():
    user_clean_timelines[user_id_str] = clean_text(user_timelines[user_id_str])

# Training

In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.models import TfidfModel
from gensim import corpora


user_timelines_keys = []
texts = []
for k, v in user_clean_timelines.items():
    user_timelines_keys.append(k)
    texts.append(v)

# filter out users with less than 200 words in the tweets in their timelines
long_timelines = [t for t in texts if len(t) > 200]
dictionary = corpora.Dictionary(long_timelines)
dictionary.filter_extremes(no_below=10, no_above=0.3, keep_n=100000)
corpus = [dictionary.doc2bow(text) for text in long_timelines]
# tfidf = TfidfModel(corpus)
# corpus = tfidf[corpus]
corpus = corpus[0:500]
print('number of user-timelines:', len(corpus))



In [ ]:
bows = list()
for doc in corpus:
    bow = dict()
    for word in doc:
        bow[dictionary[word[0]]] = word[1]
    bows.append(bow)


In [ ]:
bows = gl.SArray(bows)

if train:
    topic_model = gl.topic_model.create(bows, num_topics=100, num_iterations=200)
    topic_model.save('../models/LDA_user_timelines')

else:
    topic_model = graphlab.load_model('../models/LDA_user_timelines')

# Visualization

In [ ]:
pyLDAvis.graphlab.prepare(topic_model, bows)